# first re-capitulate how to find modules with clusters of samples 

In [1]:
import sys
import datetime
import time



from sklearn.decomposition import NMF
import numpy as np
import pandas as pd
from datetime import date
today = str(date.today())

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

BelugaMode = False
#saveDHSmat = True


# several parameters
axis_fontsize = 30
myrandom = 0
customthresh = 0.5

def mytime():
	ts = time.time()
	sttime = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d_%H:%M:%S')
	return sttime
print('starting at ', mytime(), flush=True)

def increase_axis_fontsize():
	ax = plt.gca()
	ticklabels = ax.get_xticklabels()
	for label in ticklabels:
		label.set_fontsize(axis_fontsize)
		label.set_family('serif')
	ticklabels = ax.get_yticklabels()
	for label in ticklabels:
		label.set_fontsize(axis_fontsize)
		label.set_family('serif')
		
def make_bar_plot(Nrelevant, BarMatrix, bargraph_out, plotClusterMode=False, barsortorder=[], clusterTopLabels=[], myNMFcomps=16):
	if len(barsortorder)<1:
		barsortorder = np.arange(Nrelevant)
	ttt = np.arange(Nrelevant)
	start = 0
	end = Nrelevant
	ground_pSample = ttt*0
	plt.clf()
	plt.figure(figsize=(150,40))
	plt.bar(ttt[start:end], BarMatrix[0,start:end][barsortorder], color='r',
             bottom=ground_pSample[start:end], alpha=0.75)
	ground_pSample = BarMatrix[0,start:end][barsortorder]
	for i in range(1,myNMFcomps):
		plt.bar(ttt[start:end],BarMatrix[i,start:end][barsortorder], bottom = ground_pSample, color=Comp_colors[i], alpha=0.75)
		ground_pSample = np.sum(BarMatrix[0: i+1,start:end], axis=0)[barsortorder]
	increase_axis_fontsize()
	plt.ylabel('sum of signal in matrix',fontsize=70)
	#plt.title('Full Sample',fontsize=70)
	samplenamesize = 27
	thebottom = 0.25
	if (BelugaMode):
		samplenamesize = 11
		thebottom = 0.15
	if(plotClusterMode):
		plt.xticks(ttt, ttt.astype(str), rotation='vertical', fontsize=samplenamesize)
		if len(clusterTopLabels) > 0:
			ax = plt.gca()
			ax2 = ax.twiny()
			ax2.set_xticks(ttt)
			ax2.set_xticklabels(clusterTopLabels.astype(str), rotation=90, fontsize=samplenamesize)
			#ax.xaxis.tick_top()
			#plt.xticks(ttt, clusterTopLabels.astype(str), rotation='vertical', fontsize=samplenamesize)
	else:
		plt.xticks(ttt, names[barsortorder], rotation='vertical', fontsize=samplenamesize)	
	plot_margin = 5
	plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=thebottom)
	plt.savefig(bargraph_out)
	plt.close()	

#Nsamples
if (BelugaMode):
	Nsamples = 827
	NMFcomps = 0 #will be assigned later
	B = pd.read_table('colTitles_friendly.txt', header=None)
	names = B[1].values
else:
	Nsamples = 176
	NMFcomps = 16 #has to be set here since NMF will be done on the spot
	B = pd.read_table('rowTitles.txt', header=None)
	names = B[0].values


if (BelugaMode):
	if (len(sys.argv) < 3):
		print ('syntax: blah.py pickle1(daslog) pickle2(daslog2)')
		sys.exit()
	finname1 = str(sys.argv[1])
	finname2 = str(sys.argv[2])
	print('starting read at ', mytime(), flush=True)
	A = pd.read_table('masterListPresenceAbsence_827samples_FDR0.0010_hg38.bed')
	print('finished read at ', mytime(), flush=True)
	print(A.head())
	print(A.shape)
	a = A.drop([A.columns[0], A.columns[1], A.columns[2]], axis=1).values
	a = a.T
	print('finished drop at ', mytime(), flush=True)
	del(A)
else:
	IslandSize = 400
	finname = 'mtxlist.txt'
	f = open(finname, 'r')
	dars = f.readlines()
	featurelist = []
	for line in dars:
		g = np.loadtxt(line.strip())
		#daslog = np.log10(g+1)
		daslog = (g > 0).astype(int) 
		if (daslog.shape[1] != IslandSize):
			daslog = daslog[:,:IslandSize]
		featurelist.append(daslog)
	featurelist = np.array(featurelist)
	a = np.copy(featurelist[0])
	for i in range(499):
		a= np.append(a, featurelist[i+1], axis=1)
	#print (a.shape)


if (BelugaMode):
	print('starting read NMF results at ', mytime(), flush=True)
	BasisMat = np.load(finname1)
	MixtureMat = np.load(finname2)
else:
	model = NMF(n_components=NMFcomps, init='random', random_state=0)
	BasisMat = model.fit_transform(a) 
	MixtureMat = model.components_
	print(model.reconstruction_err_)

reconstruction = np.dot(BasisMat, MixtureMat)
NMFcomps = BasisMat.shape[1]
NDHS = MixtureMat.shape[1]

Comp_colors = ['red', 'tan', 'lime','blue','m','k','c', 'coral', 'indigo','darkgreen','orange','grey','gold', 'lightskyblue', 'peru', 'olive']
if (NMFcomps>16):
	from matplotlib import colors as mcolors
	colornames = list(mcolors.CSS4_COLORS.keys())
	count = 16
	while (count < NMFcomps):
		newcolor = colornames[np.random.randint(0,len(colornames))]
		trialcount = 0
		while ((newcolor in Comp_colors) and (trialcount < 100)):
			newcolor = colornames[np.random.randint(0,len(colornames))]
			trialcount+=1
		Comp_colors.append(newcolor)
		count+=1
		
#for now lets just do unweighted version


starting at  20170630_17:03:49
1081.9722798050345


In [2]:

#calculate normed version of Sample-wise component vectors
rawnormed_Basisums = np.sum(BasisMat, axis=1)
rawnormed_VecRats = np.copy(BasisMat)
for i, rat in  enumerate(rawnormed_VecRats):
	if (rawnormed_Basisums[i]>0):
		rat/=rawnormed_Basisums[i]

#calculate normed version of DHS-wise component vectors
rawnormed_Mixturesums = np.sum(MixtureMat.T, axis=1)
rawnormed_MixtureRats = np.copy(MixtureMat.T)
rawnormed_MixtureRats = rawnormed_MixtureRats[rawnormed_Mixturesums>0]
rawnormed_Mixturesums = rawnormed_Mixturesums[rawnormed_Mixturesums>0]
for i, rat in  enumerate(rawnormed_MixtureRats):
    rat/=rawnormed_Mixturesums[i]

from sklearn.cluster import KMeans

kmeans_normed_sample_orig = KMeans(n_clusters=NMFcomps*2, random_state=0).fit(rawnormed_VecRats)
ClusterMeans = kmeans_normed_sample_orig.cluster_centers_
#this doesn't really work... i am using normed versionf of vectors and then throwing them into DHS matrix for reconstruction

ClusterPredictions = kmeans_normed_sample_orig.predict(rawnormed_VecRats)


In [34]:
ClusterPredictions

array([ 6, 10, 16, 11,  5, 10, 15,  3,  3, 30, 14,  6,  8,  7,  7,  6,  6,
       31, 29,  2,  2,  2,  1, 16, 23,  6,  2, 23, 20, 16,  8,  4, 17, 15,
       15, 17,  8,  4,  4, 25, 25,  9,  2, 20, 12,  2, 18, 25,  2, 29, 29,
        2, 17, 15,  6,  0, 11, 23,  5, 15, 23, 31, 21, 21,  9,  1, 21, 21,
       21,  6, 15, 15, 15,  1,  4, 23,  4, 23, 15, 23, 11, 17, 17, 28, 31,
        9,  2, 30,  8,  7,  7, 17,  8,  9,  9,  8, 14, 14,  0, 25, 14, 14,
        0, 14, 21, 21,  0,  7, 10, 10,  0,  0, 10,  1,  6, 31, 17,  2, 27,
       21,  3,  8, 26, 26,  5, 11, 18, 10,  4,  3,  8,  8, 20,  5,  8, 26,
       23,  4, 15, 18, 15, 31,  3,  3,  3,  1, 13, 13,  3,  3, 31, 19, 30,
        3,  3,  3, 31, 23, 24, 11, 24, 24, 24, 24, 28, 24, 19, 19,  5, 22,
       17,  4, 25,  4, 13,  4], dtype=int32)

In [39]:
#which things in which cluster

Clusters = []
# find the "cluster center" in the actual DHS decomposed space
ClusterMeansReal = []

for i in range(NMFcomps*2):
    current_cut = np.argwhere(ClusterPredictions==i).T[0]
    #print (BasisMat[current_cut].shape)
    themean = np.mean(BasisMat[current_cut], axis=0)
    #print(themean.shape)
    print(i, len(ClusterPredictions[current_cut]), names[current_cut])
    ClusterMeansReal.append(themean)
    Clusters.append([i, names[current_cut], current_cut ])


0 6 ['DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741' 'DS13368_HMF_LN2775'
 'DS13480_HCFaa_LN2800' 'DS13513_AoAF_LN2811' 'DS13573_HPdLF_LN2823']
1 5 ['DS23530_h.KBM_7_LN20483' 'DS12543_NB4_LN2556' 'DS25969A_h.CD34+_LN26316'
 'DS31642A_h.CD1_LN28530' 'DS17215_h.CD14+_LN3909']
2 10 ['DS23286_h.f.Retina_LN20434' 'DS23341_h.f.Retina_LN20451'
 'DS23368A_h.f.Retina_LN20458' 'DS23833A_h.f.Eye_LN20656'
 'DS24728A_h.f.Eye_LN21204' 'DS24796A_h.f.Eye_LN21208'
 'DS24833A_fSpinalCord_LN21289' 'DS24872A_fBrain_LN21323'
 'DS23339E_h.f.Retina_LN26813' 'DS23849C_fBrain_LN28707']
3 12 ['DS17885_CD8+_LN10088' 'DS17212_CD4+_LN10089' 'DS14108_h.Tn_LN2964'
 'DS14702_hTreg_LN3156' 'DS17189_CD56+_LN3905' 'DS17198_CD3+_LN3906'
 'DS17203_CD8+_LN3907' 'DS17597_TH2_LN4024' 'DS17603_iTH2_LN4026'
 'DS17881_CD4+_LN4097' 'DS18015_TH1_LN4128' 'DS18018_iTH1_LN4129']
4 10 ['DS24235A_h.SJSA_1_LN20839' 'DS24567A_h.Daoy_LN20970'
 'DS24640A_h.MG_63_LN21068' 'DS12829_NHLF_LN2632' 'DS26313A_h.ELR_LN26395'
 'DS14426_HSMM_LN3058' 'D

In [41]:
#old way of doing things
for i in range(NMFcomps):
    current_col = rawnormed_VecRats[:,i]
    current_cut = current_col > 0.8
    fung = np.argwhere(current_cut).T[0]
    #print(Comp_colors[i], len(rawnormed_Basisums[current_cut]), names[current_cut], fung)
    #print(Comp_colors[i], len(rawnormed_Basisums[current_cut]), names[current_cut], fung, file=f)
    #if ( len(borfRats[current_cut]) >= Num_for_cluster):
        #Clusters.append([first16colors[i], names[current_cut], fung ])
    #already_clustered[current_cut] = True


In [13]:
import scipy.spatial.distance as spdist
import matplotlib.markers as themarkers 



In [19]:
#threshold for what counts as a positive prediction in reconstruction matrix
chosenthresh = 0.5
#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
cosdist_sample_thresh = 0.3

#threshold for cosine similarity distance for DHS to be included in this module
cosdist_DHS_thresh = 0.3

# This was first attempt that didnt work out because i was using mean_of_cluster which is just 
# my normalized ratio vector... but when i do recon_cluster_DHS step it needs diff values
for i, cluster in enumerate(Clusters):

	print ('doing cluster ',i)
	# mean sample-wise component vector for this cluster
	mean_of_cluster = ClusterMeans[i]
	#create a single 1 x NDHS prediction for the cluster to
	recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)
	cluster_length = len(cluster[1])
	print('cluster length ',cluster_length)
	print('name of samples', cluster[1])
    
	#now you tile the vector to a size equivalent to N_samples_in_cluster x NDHS
	kar = np.tile(recon_cluster_DHS, cluster_length).reshape(cluster_length, NDHS)

	#***** NOTE: for some reason I decided to use original version of NMF matrices for cosine 
	#***** Similarity scores. Self-consistent? Not sure! 

	#calculate cosine similarity of cluster mean to all sample-component vectors 
	sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
	#calculate cosine similarity of cluster mean to all DHS-component vectors 
	DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
	#now we sort these sample cosine distances
	closers = np.argsort(sample_cosdists)

	#using the thresholds defined outside the loop, we find the samples that meet the closeness
	#criteria 
	Num_samples = len(sample_cosdists[sample_cosdists<cosdist_sample_thresh])
	print(' Num samples', Num_samples)
	print('samples meeting threshold ',names[closers[0:Num_samples]])

	#First we use our decision threshold to figure out if the model predicts a positive
	#for a given position. Using the sample cluster mean component vector
	DHSappear_cut = np.dot(mean_of_cluster, MixtureMat)>chosenthresh
	#now sort the coside distances of DHS's that will appear from previous step    
	DHScloseness = np.argsort(DHScosdists[DHSappear_cut])
	Num_DHSs = len(DHScosdists[DHSappear_cut][DHScosdists[DHSappear_cut] < cosdist_DHS_thresh])
	print(' Num_DHSs', Num_DHSs)

	#now we use the masks to create a miniature version of the reconstruction matrix for the given DHS only
	#strange use of double transpose 
	sorted_Module = np.dot(BasisMat[closers[0:Num_samples]], MixtureMat.T[DHSappear_cut][DHScloseness][0:Num_DHSs].T)
	#no idea how i pulled out this magic
	#this gives the real data corresponding to the module 
	corresponding_data = a[closers[0:Num_samples]][:,np.argwhere(DHSappear_cut == True).T[0][DHScloseness][0:Num_DHSs]]
	#flatten both modules to count accurate predictions. 
	flatsorted_Module = sorted_Module.flatten()
	flatcorresponding_data = corresponding_data.flatten()
	#calculate "density" i.e. number of entries of DHS hits within the module
	if (len(flatsorted_Module) > 0):
		density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
	else:
		density_in_recon = 0
	if(len(flatcorresponding_data) > 0):
		density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
	else:
		density_in_data = 0
	#calculate true positives, false positives, False negatives, precision ,recall
	TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
	if ((TP + FN ) > 0):
		recall = TP / (TP + FN)
	else: 
		recall = 0
	if ((TP +FP) > 0):
		precision = TP / (TP + FP)
	else:
		precision = 0
	print('density in reconstruction,  density in data,  precision,  recall')
	print(density_in_recon, density_in_data, precision, recall  )
	# don't remmeber how i figured this out either: I was possessed 
	#saves module as a list of samples in the header followed by a list of eahc DHS position
	#and whether it is or isn't present in the module.
	firstcut = np.copy(DHSappear_cut)
	firstcut[DHSappear_cut] *= (DHScosdists[DHSappear_cut] < cosdist_DHS_thresh)
	list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
	theheader =  ' '.join(str(e) for e in list1)
	namestr = ' '.join(names[closers[0:Num_samples]])
	theheader +=' '+namestr
	#foutname = today+'Module'+str(i)+'.txt'
	#np.savetxt(foutname, firstcut, fmt='%i', header=theheader)
	#'''

doing cluster  0
cluster length  6
name of samples ['DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741' 'DS13368_HMF_LN2775'
 'DS13480_HCFaa_LN2800' 'DS13513_AoAF_LN2811' 'DS13573_HPdLF_LN2823']
 Num samples 22
 Num_DHSs 1259
samples meeting threshold  ['DS13368_HMF_LN2775' 'DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741'
 'DS13573_HPdLF_LN2823' 'DS13513_AoAF_LN2811' 'DS12684_HIPEpiC_LN2583'
 'DS13480_HCFaa_LN2800' 'DS34574A_h.CEC_LN29827' 'DS11949_SKMC_LN2373'
 'DS14790_HAsp_LN3184' 'DS21042_h.H1.polyMSC_LN10043' 'DS13411_HPAF_LN2784'
 'DS12447_HCPEpiC_LN2530' 'DS12501_HCF_LN2542' 'DS13981_HVMF_LN2925'
 'DS12663_HAEpiC_LN2579' 'DS12599_HCM_LN2565' 'DS13390_HPF_LN2780'
 'DS14426_HSMM_LN3058' 'DS20548_LHCN_M2_LN4870' 'DS12863_NHDF_Ad_LN2643'
 'DS12374_AG10803_LN2497']
density in reconstruction,  density in data,  precision,  recall
0.9942595133222616 0.915733988013575 0.9190602418388467 0.9978709982652578
doing cluster  1
cluster length  5
name of samples ['DS23530_h.KBM_7_LN20483' 'DS12543_NB4_LN25

In [40]:
#using ClusterMeans Real (actual values from newdaslog)

#threshold for what counts as a positive prediction in reconstruction matrix
chosenthresh = 0.5
#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
cosdist_sample_thresh = 0.3

#threshold for cosine similarity distance for DHS to be included in this module
cosdist_DHS_thresh = 0.3
for i, cluster in enumerate(Clusters):

	print ('doing cluster ',i)
	# mean sample-wise component vector for this cluster
	mean_of_cluster = ClusterMeansReal[i]
	#create a single 1 x NDHS prediction for the cluster to
	recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)
	cluster_length = len(cluster[1])
	print('cluster length ',cluster_length)
	print('name of samples', cluster[1])
    
	#now you tile the vector to a size equivalent to N_samples_in_cluster x NDHS
	kar = np.tile(recon_cluster_DHS, cluster_length).reshape(cluster_length, NDHS)

	#***** NOTE: for some reason I decided to use original version of NMF matrices for cosine 
	#***** Similarity scores. Self-consistent? Not sure! 

	#calculate cosine similarity of cluster mean to all sample-component vectors 
	sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
	#calculate cosine similarity of cluster mean to all DHS-component vectors 
	DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
	#now we sort these sample cosine distances
	closers = np.argsort(sample_cosdists)

	#using the thresholds defined outside the loop, we find the samples that meet the closeness
	#criteria 
	Num_samples = len(sample_cosdists[sample_cosdists<cosdist_sample_thresh])
	print(' Num samples', Num_samples)
	print('samples meeting threshold ',names[closers[0:Num_samples]])

	#First we use our decision threshold to figure out if the model predicts a positive
	#for a given position. Using the sample cluster mean component vector
	DHSappear_cut = np.dot(mean_of_cluster, MixtureMat)>chosenthresh
	#now sort the coside distances of DHS's that will appear from previous step    
	DHScloseness = np.argsort(DHScosdists[DHSappear_cut])
	Num_DHSs = len(DHScosdists[DHSappear_cut][DHScosdists[DHSappear_cut] < cosdist_DHS_thresh])
	print(' Num_DHSs', Num_DHSs)

	#now we use the masks to create a miniature version of the reconstruction matrix for the given DHS only
	#strange use of double transpose 
	sorted_Module = np.dot(BasisMat[closers[0:Num_samples]], MixtureMat.T[DHSappear_cut][DHScloseness][0:Num_DHSs].T)
	#no idea how i pulled out this magic
	#this gives the real data corresponding to the module 
	corresponding_data = a[closers[0:Num_samples]][:,np.argwhere(DHSappear_cut == True).T[0][DHScloseness][0:Num_DHSs]]
	#flatten both modules to count accurate predictions. 
	flatsorted_Module = sorted_Module.flatten()
	flatcorresponding_data = corresponding_data.flatten()
	#calculate "density" i.e. number of entries of DHS hits within the module
	if (len(flatsorted_Module) > 0):
		density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
	else:
		density_in_recon = 0
	if(len(flatcorresponding_data) > 0):
		density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
	else:
		density_in_data = 0
	#calculate true positives, false positives, False negatives, precision ,recall
	TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
	if ((TP + FN ) > 0):
		recall = TP / (TP + FN)
	else: 
		recall = 0
	if ((TP +FP) > 0):
		precision = TP / (TP + FP)
	else:
		precision = 0
	print('density in reconstruction,  density in data,  precision,  recall')
	print(density_in_recon, density_in_data, precision, recall  )
	# don't remmeber how i figured this out either: I was possessed 
	#saves module as a list of samples in the header followed by a list of eahc DHS position
	#and whether it is or isn't present in the module.
	firstcut = np.copy(DHSappear_cut)
	firstcut[DHSappear_cut] *= (DHScosdists[DHSappear_cut] < cosdist_DHS_thresh)
	list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
	theheader =  ' '.join(str(e) for e in list1)
	namestr = ' '.join(names[closers[0:Num_samples]])
	theheader +=' '+namestr
	#foutname = today+'Module'+str(i)+'.txt'
	#np.savetxt(foutname, firstcut, fmt='%i', header=theheader)
	#'''

doing cluster  0
cluster length  6
name of samples ['DS11752_HGF_LN2307' 'DS13219_IMR90_LN2741' 'DS13368_HMF_LN2775'
 'DS13480_HCFaa_LN2800' 'DS13513_AoAF_LN2811' 'DS13573_HPdLF_LN2823']
samples meeting threshold  []
 Num samples 22
 Num_DHSs 2314
density in reconstruction,  density in data,  precision,  recall
0.9058301249312485 0.7871061522746916 0.8328707117144468 0.9584976291489893
doing cluster  1
cluster length  5
name of samples ['DS23530_h.KBM_7_LN20483' 'DS12543_NB4_LN2556' 'DS25969A_h.CD34+_LN26316'
 'DS31642A_h.CD1_LN28530' 'DS17215_h.CD14+_LN3909']
samples meeting threshold  ['DS12543_NB4_LN2556' 'DS23530_h.KBM_7_LN20483' 'DS31642A_h.CD1_LN28530'
 'DS16924_K562_LN3817' 'DS12274_h.CD34+_LN2482' 'DS24094A_h.CD34+_LN20866'
 'DS24044A_h.CD34+_LN20861' 'DS17215_h.CD14+_LN3909'
 'DS25969A_h.CD34+_LN26316' 'DS24074A_h.CD34+_LN21381'
 'DS25769A_h.CD34+_LN26258' 'DS15363_K562_LN3327'
 'DS25730A_h.CD34+_LN26288' 'DS25687A_h.CD34+_LN26240'
 'DS12734_h.CD34+_LN10087' 'DS26444A_h.CD34+_

# now find modules starting from DHS clusters

In [20]:
#reminder of how this works
'''
rawnormed_Mixturesums = np.sum(MixtureMat.T, axis=1)
rawnormed_MixtureRats = np.copy(MixtureMat.T)
rawnormed_MixtureRats = rawnormed_MixtureRats[rawnormed_Mixturesums>0]
rawnormed_Mixturesums = rawnormed_Mixturesums[rawnormed_Mixturesums>0]
for i, rat in  enumerate(rawnormed_MixtureRats):
    rat/=rawnormed_Mixturesums[i]

from sklearn.cluster import KMeans

kmeans_normed_sample_orig = KMeans(n_clusters=NMFcomps*2, random_state=0).fit(rawnormed_VecRats)
ClusterMeans = kmeans_normed_sample_orig.cluster_centers_
ClusterPredictions = kmeans_normed_sample_orig.predict(rawnormed_VecRats)
'''

'\nrawnormed_Mixturesums = np.sum(MixtureMat.T, axis=1)\nrawnormed_MixtureRats = np.copy(MixtureMat.T)\nrawnormed_MixtureRats = rawnormed_MixtureRats[rawnormed_Mixturesums>0]\nrawnormed_Mixturesums = rawnormed_Mixturesums[rawnormed_Mixturesums>0]\nfor i, rat in  enumerate(rawnormed_MixtureRats):\n    rat/=rawnormed_Mixturesums[i]\n\nfrom sklearn.cluster import KMeans\n\nkmeans_normed_sample_orig = KMeans(n_clusters=NMFcomps*2, random_state=0).fit(rawnormed_VecRats)\nClusterMeans = kmeans_normed_sample_orig.cluster_centers_\nClusterPredictions = kmeans_normed_sample_orig.predict(rawnormed_VecRats)\n'

In [21]:
kmeans_normed_DHS_orig = KMeans(n_clusters=NMFcomps*2, random_state=0).fit(rawnormed_MixtureRats)

In [22]:
ClusterMeansDHS = kmeans_normed_DHS_orig.cluster_centers_
ClusterPredictionsDHS = kmeans_normed_DHS_orig.predict(rawnormed_MixtureRats)

In [25]:
ClusterPredictionsDHS.shape

(142969,)

In [26]:
ClusterMeansDHS.shape

(32, 16)

In [42]:
ClustersDHS = []
ClusterMeanRealDHS = []
for i in range(NMFcomps*2):
    current_cut = np.argwhere(ClusterPredictionsDHS==i).T[0]
    themean = np.mean(MixtureMat.T[current_cut], axis=0)
    ClusterMeanRealDHS.append(themean)
    print(len(current_cut), themean.shape)
    print(i, len(ClusterPredictionsDHS[current_cut]), current_cut)
    #to mimic sample names, I'm just using the second element to "name" DHS's
    ClustersDHS.append([i, current_cut.astype(str), current_cut])

8502 (16,)
0 8502 [    31     32     33 ..., 142390 142392 142393]
5553 (16,)
1 5553 [  2226   2227   2244 ..., 142728 142825 142826]
3641 (16,)
2 3641 [  1166   1167   1168 ..., 142949 142967 142968]
2886 (16,)
3 2886 [  1463   1464   1477 ..., 142752 142753 142754]
11815 (16,)
4 11815 [   206    207    208 ..., 142510 142511 142512]
3434 (16,)
5 3434 [     0      1     52 ..., 142449 142450 142451]
4141 (16,)
6 4141 [    11     12     13 ..., 142403 142404 142405]
3374 (16,)
7 3374 [   739    740    741 ..., 141501 141502 142367]
2954 (16,)
8 2954 [  1249   1307   1308 ..., 142850 142915 142916]
2371 (16,)
9 2371 [   611    612    613 ..., 142227 142368 142373]
3789 (16,)
10 3789 [   933    934    935 ..., 142166 142167 142168]
12052 (16,)
11 12052 [   175    177    178 ..., 142930 142931 142932]
4689 (16,)
12 4689 [  1124   1164   1172 ..., 142396 142397 142398]
4079 (16,)
13 4079 [   319    320    327 ..., 142520 142521 142522]
1541 (16,)
14 1541 [  5826   5827   5828 ..., 142067 1

In [55]:
#threshold for what counts as a positive prediction in reconstruction matrix
chosenthresh = 0.5
#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
cosdist_sample_thresh = 0.5

#threshold for cosine similarity distance for DHS to be included in this module
cosdist_DHS_thresh = 0.3
for i, cluster in enumerate(ClustersDHS):

	print ('doing cluster ',i)
	# mean sample-wise component vector for this cluster
	mean_of_cluster = ClusterMeanRealDHS[i]
	#create a single 1 x NDHS prediction for the cluster to
	recon_cluster_Sam = np.dot(BasisMat, mean_of_cluster)
	#recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)

	cluster_length = len(cluster[1])
	print('cluster length ',cluster_length)
	print('name of samples', cluster[1])
    
	#now you tile the vector to a size equivalent to N_DHS_in_cluster x NSamples
	kar = np.tile(recon_cluster_Sam, cluster_length).reshape(cluster_length, Nsamples)
	print(kar.shape)
	kar = kar.T
    
	#print(kar[0]-kar[1])    
	#***** NOTE: for some reason I decided to use original version of NMF matrices for cosine 
	#***** Similarity scores. Self-consistent? Not sure! 

	#calculate cosine similarity of cluster mean to all sample-component vectors 
	sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
	#calculate cosine similarity of cluster mean to all DHS-component vectors 
	DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
	#now we sort these sample cosine distances

	closers = np.argsort(DHScosdists)

	#using the thresholds defined outside the loop, we find the samples that meet the closeness
	#criteria 
	Num_DHSs = len(DHScosdists[DHScosdists<cosdist_sample_thresh])
	print(' Num DHSs', Num_DHSs)
	print('DHSs meeting threshold ',closers[0:Num_DHSs])
	#First we use our decision threshold to figure out if the model predicts a positive
	#for a given position. Using the sample cluster mean component vector
	Sample_appear_cut = np.dot(mean_of_cluster, BasisMat.T)>chosenthresh
	#now sort the coside distances of DHS's that will appear from previous step    

	Sample_closeness = np.argsort(sample_cosdists[Sample_appear_cut])
	print('close samples ',sample_cosdists[Sample_appear_cut])
	Num_samples = len(sample_cosdists[Sample_appear_cut][sample_cosdists[Sample_appear_cut] < cosdist_sample_thresh])
	print(' Num_samples', Num_samples)

	'''
	#now we use the masks to create a miniature version of the reconstruction matrix for the given DHS only
	#strange use of double transpose 
	sorted_Module = np.dot(BasisMat[closers[0:Num_samples]], MixtureMat.T[DHSappear_cut][DHScloseness][0:Num_DHSs].T)
	#no idea how i pulled out this magic
	#this gives the real data corresponding to the module 
	corresponding_data = a[closers[0:Num_samples]][:,np.argwhere(DHSappear_cut == True).T[0][DHScloseness][0:Num_DHSs]]
	#flatten both modules to count accurate predictions. 
	flatsorted_Module = sorted_Module.flatten()
	flatcorresponding_data = corresponding_data.flatten()
	#calculate "density" i.e. number of entries of DHS hits within the module
	if (len(flatsorted_Module) > 0):
		density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
	else:
		density_in_recon = 0
	if(len(flatcorresponding_data) > 0):
		density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
	else:
		density_in_data = 0
	#calculate true positives, false positives, False negatives, precision ,recall
	TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
	FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
	if ((TP + FN ) > 0):
		recall = TP / (TP + FN)
	else: 
		recall = 0
	if ((TP +FP) > 0):
		precision = TP / (TP + FP)
	else:
		precision = 0
	print('density in reconstruction,  density in data,  precision,  recall')
	print(density_in_recon, density_in_data, precision, recall  )
	# don't remmeber how i figured this out either: I was possessed 
	#saves module as a list of samples in the header followed by a list of eahc DHS position
	#and whether it is or isn't present in the module.
	firstcut = np.copy(DHSappear_cut)
	firstcut[DHSappear_cut] *= (DHScosdists[DHSappear_cut] < cosdist_DHS_thresh)
	list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
	theheader =  ' '.join(str(e) for e in list1)
	namestr = ' '.join(names[closers[0:Num_samples]])
	theheader +=' '+namestr
	#foutname = today+'Module'+str(i)+'.txt'
	#np.savetxt(foutname, firstcut, fmt='%i', header=theheader)
	'''

doing cluster  0
cluster length  8502
name of samples ['31' '32' '33' ..., '142390' '142392' '142393']
(8502, 176)
 Num DHSs 60448
DHSs meeting threshold  [ 82968  82958  29503 ..., 135436 135435  64646]
close samples  []
 Num_samples 0
doing cluster  1
cluster length  5553
name of samples ['2226' '2227' '2244' ..., '142728' '142825' '142826']
(5553, 176)
 Num DHSs 63205
DHSs meeting threshold  [ 82969  82957 157485 ...,  64224  64227 185909]
close samples  []
 Num_samples 0
doing cluster  2
cluster length  3641
name of samples ['1166' '1167' '1168' ..., '142949' '142967' '142968']
(3641, 176)
 Num DHSs 65898
DHSs meeting threshold  [ 82968 158156  66488 ..., 150950 185846  27334]
close samples  []
 Num_samples 0
doing cluster  3
cluster length  2886
name of samples ['1463' '1464' '1477' ..., '142752' '142753' '142754']
(2886, 176)
 Num DHSs 61987
DHSs meeting threshold  [ 82968  71096  47064 ..., 126811 126813 139813]
close samples  []
 Num_samples 0
doing cluster  4
cluster length  1

# confused as to why there are no hits so far 
# breaking down what this loop is doing

In [65]:
i = 1
cluster = ClustersDHS[1] 

In [66]:
i

1

In [67]:
cluster

[1, array(['2226', '2227', '2244', ..., '142728', '142825', '142826'], 
       dtype='<U21'), array([  2226,   2227,   2244, ..., 142728, 142825, 142826])]

In [68]:
mean_of_cluster = ClusterMeanRealDHS[i]


In [69]:
mean_of_cluster

array([ 0.05822718,  0.06602308,  0.07433184,  0.09739714,  0.08120447,
        0.0742244 ,  0.12758088,  0.05706969,  0.04158215,  0.04890431,
        0.04539347,  0.05544609,  0.04930681,  0.04954408,  0.04463254,
        0.03677961])

In [70]:
ClusterMeansDHS[i]

array([ 0.01198427,  0.00371326,  0.00688537,  0.01065807,  0.00209255,
        0.0054197 ,  0.86434062,  0.00405588,  0.00183566,  0.0137023 ,
        0.00205556,  0.00882534,  0.00344329,  0.01664763,  0.00177357,
        0.04256691])

So using the normalization method actually brings out DHS component prominence significantly. (look at component [3]) 

In [71]:
np.sum(mean_of_cluster)

1.0076477354396465

strangely the mean of the cluster is earily clsoe too 1

In [72]:
recon_cluster_Sam = np.dot(BasisMat, mean_of_cluster)
recon_cluster_Sam

array([ 0.1020025 ,  0.07768347,  0.05108312,  0.08168151,  0.12938347,
        0.18651251,  0.14132165,  0.12187916,  0.13372164,  0.11913432,
        0.11390353,  0.08655075,  0.07204568,  0.10000272,  0.10128194,
        0.12112767,  0.11283353,  0.09136   ,  0.11300922,  0.08389669,
        0.15535111,  0.15129931,  0.12088458,  0.05239591,  0.131231  ,
        0.1010999 ,  0.17291445,  0.11526417,  0.10263092,  0.02892085,
        0.09485474,  0.10932463,  0.12720252,  0.10478395,  0.11363785,
        0.08120893,  0.10497748,  0.10982251,  0.08293941,  0.05359996,
        0.05656346,  0.14840515,  0.1630957 ,  0.11769693,  0.10791017,
        0.13920757,  0.09096403,  0.07185517,  0.12589026,  0.11741236,
        0.12384789,  0.10295215,  0.08686159,  0.13181881,  0.08467922,
        0.08165411,  0.08646335,  0.10433852,  0.11890637,  0.1170187 ,
        0.09001719,  0.09557137,  0.1045769 ,  0.10414237,  0.12661849,
        0.11804146,  0.11192252,  0.09648954,  0.0966603 ,  0.10

In [75]:
len(recon_cluster_Sam[recon_cluster_Sam>0.15])

8

In [77]:
cluster_length = len(cluster[1])
print('cluster length ',cluster_length)
print('name of samples', cluster[1])


cluster length  5553
name of samples ['2226' '2227' '2244' ..., '142728' '142825' '142826']


In [78]:
#now you tile the vector to a size equivalent to N_samples_in_cluster x NDHS
kar = np.tile(recon_cluster_Sam, cluster_length).reshape(cluster_length, Nsamples)
print(kar.shape)
kar = kar.T
    

(5553, 176)


In [80]:
kar.shape

(176, 5553)

In [81]:
sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
#calculate cosine similarity of cluster mean to all DHS-component vectors 
DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
#now we sort these sample cosine distances


In [84]:
DHScosdists[np.isfinite(DHScosdists)].shape

(142969,)

In [85]:
sample_cosdists

array([ 0.66023464,  0.64214774,  0.60960991,  0.6313073 ,  0.4697448 ,
        0.72574436,  0.72395996,  0.48572436,  0.44873567,  0.71574305,
        0.64409009,  0.70213946,  0.7923714 ,  0.86282019,  0.86282019,
        0.69338896,  0.68491382,  0.72093851,  0.52245613,  0.75253004,
        0.78030437,  0.77184004,  0.73123654,  0.67606646,  0.73817555,
        0.68747761,  0.74340242,  0.70594501,  0.83281174,  0.7828255 ,
        0.7029652 ,  0.56435293,  0.65629252,  0.74273023,  0.80794105,
        0.65609234,  0.69901341,  0.47131141,  0.5850186 ,  0.64180928,
        0.62794095,  0.80292066,  0.76036299,  0.82966454,  0.55059091,
        0.76528702,  0.57244301,  0.66227351,  0.74639847,  0.63586032,
        0.62470417,  0.73581688,  0.61109028,  0.78877669,  0.66878174,
        0.67128335,  0.59799629,  0.68407811,  0.6467219 ,  0.7366177 ,
        0.72883049,  0.63146457,  0.68936733,  0.68113172,  0.80351232,
        0.70881167,  0.65543328,  0.70035655,  0.64720747,  0.67

In [86]:
np.sort(sample_cosdists)

array([ 0.35897979,  0.39491857,  0.44873567,  0.46080149,  0.46371896,
        0.46898003,  0.4697448 ,  0.47131141,  0.47234841,  0.4736471 ,
        0.47693602,  0.48382687,  0.48572436,  0.48718373,  0.48934161,
        0.48956579,  0.49092178,  0.50431848,  0.50516431,  0.50837894,
        0.51695646,  0.51862157,  0.52245613,  0.53454178,  0.53590634,
        0.54091068,  0.55059091,  0.56435293,  0.56676086,  0.57244301,
        0.57639181,  0.58132478,  0.58400716,  0.5850186 ,  0.58678197,
        0.5879326 ,  0.589302  ,  0.58997932,  0.59368317,  0.5947704 ,
        0.59799629,  0.59946219,  0.59954403,  0.60547978,  0.60960991,
        0.61042821,  0.61109028,  0.61319182,  0.61571018,  0.62165964,
        0.62470417,  0.62794095,  0.63073305,  0.6313073 ,  0.63146457,
        0.6335019 ,  0.63586032,  0.63814489,  0.64180928,  0.64214774,
        0.64409009,  0.64440034,  0.64603718,  0.6467219 ,  0.64720747,
        0.6490663 ,  0.65047233,  0.65175623,  0.65536663,  0.65

In [88]:
np.sort(DHScosdists[np.isfinite(DHScosdists)])[0:50]

array([ 0.00708659,  0.01304132,  0.01440481,  0.0153048 ,  0.01538528,
        0.01602576,  0.01631966,  0.01720278,  0.01737159,  0.0180378 ,
        0.01827807,  0.01927728,  0.01929385,  0.01929385,  0.01930549,
        0.01930549,  0.01948113,  0.01984151,  0.01986077,  0.02002447,
        0.02010554,  0.02016422,  0.02017137,  0.02022556,  0.02027348,
        0.02032024,  0.02045124,  0.02061076,  0.02082312,  0.0208321 ,
        0.02086567,  0.0208923 ,  0.02102452,  0.0214732 ,  0.0214732 ,
        0.02151757,  0.02166665,  0.02170902,  0.02170902,  0.02170902,
        0.02171756,  0.02171756,  0.02171756,  0.02171756,  0.02171866,
        0.02189743,  0.02192897,  0.0220662 ,  0.02210256,  0.02235327])

So there are SOME DHS within 0.05 and some samples within 0.5  if we use even the crappy unnormalized cluster

In [89]:
closers = np.argsort(DHScosdists)


In [90]:
closers

array([ 82969,  82957, 157485, ..., 100772, 100780, 199999])

In [92]:
DHScosdists[closers]

array([ 0.00708659,  0.01304132,  0.01440481, ...,         nan,
               nan,         nan])

In [93]:
Num_DHSs = len(DHScosdists[DHScosdists<cosdist_sample_thresh])


In [94]:
Num_DHSs

63205

In [106]:
chosenthresh = 0.15

Has to be this low or it won't work at all

In [107]:
Sample_appear_cut = np.dot(mean_of_cluster, BasisMat.T)>chosenthresh


In [108]:
Sample_appear_cut.shape

(176,)

In [109]:
Sample_appear_cut[Sample_appear_cut]

array([ True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [111]:
cosdist_sample_thresh = 0.75

In [112]:
Sample_closeness = np.argsort(sample_cosdists[Sample_appear_cut])
print('close samples ',sample_cosdists[Sample_appear_cut])
Num_samples = len(sample_cosdists[Sample_appear_cut][sample_cosdists[Sample_appear_cut] < cosdist_sample_thresh])
print(' Num_samples', Num_samples)


close samples  [ 0.72574436  0.78030437  0.77184004  0.74340242  0.76036299  0.79862688
  0.79220179  0.68635864]
 Num_samples 3


yep this sucks too. Doesn't work unless i set ridiculously high cosdist sample threshold

In [117]:
BasisMat[Sample_appear_cut][Sample_closeness][0:Num_samples]

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   2.12094960e-02,   1.86337961e+00,
          1.45197911e-02,   6.60361815e-02,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   2.20981430e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.05229999e-03,   1.99095307e-02,   4.89285891e-02,
          4.73894941e-02,   3.98663875e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   2.85574787e-01,
          2.83717562e-01,   2.47140222e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   4.66178787e-02,   4.25898477e-02,
          3.43361694e-03,   0.00000000e+00,   1.49222180e-01,
          0.00000000e+00,   0.00000000e+00,   1.20157780e-01,
          0.00000000e+00,   2.09308827e-02,   2.50228652e+00,
          8.46569517e-02,   6.94571541e-02,   4.02350945e-02,
          3.1560

In [120]:
BasisMat[Sample_appear_cut][Sample_closeness][0:Num_samples].shape

(3, 16)

In [119]:
MixtureMat.T[closers][0:Num_DHSs].shape

(63205, 16)

In [123]:
sorted_Module = np.dot(BasisMat[Sample_appear_cut][Sample_closeness][0:Num_samples], MixtureMat.T[closers][0:Num_DHSs].T)

In [124]:
sorted_Module.shape

(3, 63205)

In [125]:
corresponding_data = a[Sample_appear_cut][Sample_closeness][0:Num_samples][:,np.arange(NDHS)[closers][0:Num_DHSs]]


In [126]:
corresponding_data.shape

(3, 63205)

In [127]:
flatsorted_Module = sorted_Module.flatten()
flatcorresponding_data = corresponding_data.flatten()
#calculate "density" i.e. number of entries of DHS hits within the module
if (len(flatsorted_Module) > 0):
    density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
else:
    density_in_recon = 0
if(len(flatcorresponding_data) > 0):
    density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
else:
    density_in_data = 0
#calculate true positives, false positives, False negatives, precision ,recall
TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
if ((TP + FN ) > 0):
    recall = TP / (TP + FN)
else: 
    recall = 0
if ((TP +FP) > 0):
    precision = TP / (TP + FP)
else:
    precision = 0
print('density in reconstruction,  density in data,  precision,  recall')
print(density_in_recon, density_in_data, precision, recall  )

density in reconstruction,  density in data,  precision,  recall
0.5774912322337368 0.4168024681591646 0.7184409274801143 0.9954195768802511


In [128]:
firstcut = np.zeros(NDHS)
firstcut[closers][0:Num_DHSs] += 1

In [129]:
list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]


In [131]:
theheader =  ' '.join(str(e) for e in list1)
namestr = ' '.join(names[Sample_appear_cut][Sample_closeness][0:Num_samples])
theheader +=' '+namestr

In [132]:
namestr

'DS17767_fMuscle_back_LN4072 DS20346_fPlacenta_LN10066 DS23833A_h.f.Eye_LN20656'

# trying again via loop (using real DHS centroids) 

In [133]:
chosenthresh = 0.15
#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
cosdist_sample_thresh = 0.75

#threshold for cosine similarity distance for DHS to be included in this module
cosdist_DHS_thresh = 0.3
for i, cluster in enumerate(ClustersDHS):

    print ('doing cluster ',i)
    # mean sample-wise component vector for this cluster
    mean_of_cluster = ClusterMeanRealDHS[i]
    #create a single 1 x NDHS prediction for the cluster to
    recon_cluster_Sam = np.dot(BasisMat, mean_of_cluster)
    #recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)

    cluster_length = len(cluster[1])
    print('cluster length ',cluster_length)
    print('name of samples', cluster[1])

    #now you tile the vector to a size equivalent to N_DHS_in_cluster x NSamples
    kar = np.tile(recon_cluster_Sam, cluster_length).reshape(cluster_length, Nsamples)
    print(kar.shape)
    kar = kar.T

    #print(kar[0]-kar[1])    
    #***** NOTE: for some reason I decided to use original version of NMF matrices for cosine 
    #***** Similarity scores. Self-consistent? Not sure! 

    #calculate cosine similarity of cluster mean to all sample-component vectors 
    sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
    #calculate cosine similarity of cluster mean to all DHS-component vectors 
    DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
    #now we sort these sample cosine distances

    closers = np.argsort(DHScosdists)

    #using the thresholds defined outside the loop, we find the samples that meet the closeness
    #criteria 
    Num_DHSs = len(DHScosdists[DHScosdists<cosdist_sample_thresh])
    print(' Num DHSs', Num_DHSs)
    print('DHSs meeting threshold ',closers[0:Num_DHSs])
    #First we use our decision threshold to figure out if the model predicts a positive
    #for a given position. Using the sample cluster mean component vector
    Sample_appear_cut = np.dot(mean_of_cluster, BasisMat.T)>chosenthresh
    #now sort the coside distances of DHS's that will appear from previous step    

    Sample_closeness = np.argsort(sample_cosdists[Sample_appear_cut])
    print('close samples ',sample_cosdists[Sample_appear_cut])
    Num_samples = len(sample_cosdists[Sample_appear_cut][sample_cosdists[Sample_appear_cut] < cosdist_sample_thresh])
    print(' Num_samples', Num_samples)

    sorted_Module = np.dot(BasisMat[Sample_appear_cut][Sample_closeness][0:Num_samples], MixtureMat.T[closers][0:Num_DHSs].T)
    corresponding_data = a[Sample_appear_cut][Sample_closeness][0:Num_samples][:,np.arange(NDHS)[closers][0:Num_DHSs]]
    flatsorted_Module = sorted_Module.flatten()
    flatcorresponding_data = corresponding_data.flatten()
    #calculate "density" i.e. number of entries of DHS hits within the module
    if (len(flatsorted_Module) > 0):
        density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
    else:
        density_in_recon = 0
    if(len(flatcorresponding_data) > 0):
        density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
    else:
        density_in_data = 0
    #calculate true positives, false positives, False negatives, precision ,recall
    TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
    FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
    FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
    if ((TP + FN ) > 0):
        recall = TP / (TP + FN)
    else: 
        recall = 0
    if ((TP +FP) > 0):
        precision = TP / (TP + FP)
    else:
        precision = 0
    print('density in reconstruction,  density in data,  precision,  recall')
    print(density_in_recon, density_in_data, precision, recall  )
    firstcut = np.zeros(NDHS)
    firstcut[closers][0:Num_DHSs] += 1
    list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
    theheader =  ' '.join(str(e) for e in list1)
    namestr = ' '.join(names[Sample_appear_cut][Sample_closeness][0:Num_samples])
    theheader +=' '+namestr
    print(namestr)

doing cluster  0
cluster length  8502
name of samples ['31' '32' '33' ..., '142390' '142392' '142393']
(8502, 176)
 Num DHSs 127240
DHSs meeting threshold  [ 82968  82958  29503 ..., 123761  67434  53861]
close samples  [ 0.47409042  0.77938278  0.69272076  0.81374756  0.80374952  0.73376755
  0.7758224   0.82396248  0.79187757  0.80070422  0.61490029  0.65169914
  0.58886075  0.82113003  0.83858314  0.80880186  0.57242276  0.67909595
  0.69645321  0.53516453  0.64115807  0.69393815  0.71021521]
 Num_samples 13
density in reconstruction,  density in data,  precision,  recall
0.3684587575266607 0.23255991101008391 0.5978243570285902 0.9471693502685334
DS21042_h.H1.polyMSC_LN10043 DS17878_fStomach_LN4096 DS15192_HAh_LN3285 DS26410A_h.AL_LN26455 DS24888A_h.f.Tongue_LN21292 DS18224_Skin_fibroblast_LN4158 DS26113A_h.Fibroblast_LN26368 DS17186_CD19+_LN3904 DS20786_fKidney_LN10113 DS20239_fMuscle_leg_LN4772 DS17767_fMuscle_back_LN4072 DS20242_fMuscle_trunk_LN4773 DS23559_h.EH_LN20491
doing cl

# now even though it is in consistent, lets use the normalized DHS vectors for cluster means

In [134]:
chosenthresh = 0.3
#threshold for cosine similarity distance for samples to be included in this module. 0.1 gives about the same samples as in the original cluster for 80% component dominance. 
cosdist_sample_thresh = 0.5

#threshold for cosine similarity distance for DHS to be included in this module
cosdist_DHS_thresh = 0.3
for i, cluster in enumerate(ClustersDHS):

    print ('doing cluster ',i)
    # mean sample-wise component vector for this cluster
    mean_of_cluster = ClusterMeansDHS[i]
    #create a single 1 x NDHS prediction for the cluster to
    recon_cluster_Sam = np.dot(BasisMat, mean_of_cluster)
    #recon_cluster_DHS = np.dot(mean_of_cluster, MixtureMat)

    cluster_length = len(cluster[1])
    print('cluster length ',cluster_length)
    print('name of samples', cluster[1])

    #now you tile the vector to a size equivalent to N_DHS_in_cluster x NSamples
    kar = np.tile(recon_cluster_Sam, cluster_length).reshape(cluster_length, Nsamples)
    print(kar.shape)
    kar = kar.T

    #print(kar[0]-kar[1])    
    #***** NOTE: for some reason I decided to use original version of NMF matrices for cosine 
    #***** Similarity scores. Self-consistent? Not sure! 

    #calculate cosine similarity of cluster mean to all sample-component vectors 
    sample_cosdists = spdist.cdist(BasisMat, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()    
    #calculate cosine similarity of cluster mean to all DHS-component vectors 
    DHScosdists = spdist.cdist(MixtureMat.T, np.reshape(mean_of_cluster, (1,NMFcomps)), 'cosine').flatten()
    #now we sort these sample cosine distances

    closers = np.argsort(DHScosdists)

    #using the thresholds defined outside the loop, we find the samples that meet the closeness
    #criteria 
    Num_DHSs = len(DHScosdists[DHScosdists<cosdist_sample_thresh])
    print(' Num DHSs', Num_DHSs)
    print('DHSs meeting threshold ',closers[0:Num_DHSs])
    #First we use our decision threshold to figure out if the model predicts a positive
    #for a given position. Using the sample cluster mean component vector
    Sample_appear_cut = np.dot(mean_of_cluster, BasisMat.T)>chosenthresh
    #now sort the coside distances of DHS's that will appear from previous step    

    Sample_closeness = np.argsort(sample_cosdists[Sample_appear_cut])
    print('close samples ',sample_cosdists[Sample_appear_cut])
    Num_samples = len(sample_cosdists[Sample_appear_cut][sample_cosdists[Sample_appear_cut] < cosdist_sample_thresh])
    print(' Num_samples', Num_samples)

    sorted_Module = np.dot(BasisMat[Sample_appear_cut][Sample_closeness][0:Num_samples], MixtureMat.T[closers][0:Num_DHSs].T)
    corresponding_data = a[Sample_appear_cut][Sample_closeness][0:Num_samples][:,np.arange(NDHS)[closers][0:Num_DHSs]]
    flatsorted_Module = sorted_Module.flatten()
    flatcorresponding_data = corresponding_data.flatten()
    #calculate "density" i.e. number of entries of DHS hits within the module
    if (len(flatsorted_Module) > 0):
        density_in_recon = len(flatsorted_Module[flatsorted_Module>chosenthresh])/len(flatsorted_Module)
    else:
        density_in_recon = 0
    if(len(flatcorresponding_data) > 0):
        density_in_data = len(flatcorresponding_data[flatcorresponding_data>chosenthresh])/len(flatcorresponding_data)
    else:
        density_in_data = 0
    #calculate true positives, false positives, False negatives, precision ,recall
    TP =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
    FP =len(flatcorresponding_data[ np.invert(flatcorresponding_data>chosenthresh) * (flatsorted_Module>chosenthresh)])
    FN =len(flatcorresponding_data[ (flatcorresponding_data>chosenthresh) * np.invert(flatsorted_Module>chosenthresh)])
    if ((TP + FN ) > 0):
        recall = TP / (TP + FN)
    else: 
        recall = 0
    if ((TP +FP) > 0):
        precision = TP / (TP + FP)
    else:
        precision = 0
    print('density in reconstruction,  density in data,  precision,  recall')
    print(density_in_recon, density_in_data, precision, recall  )
    firstcut = np.zeros(NDHS)
    firstcut[closers][0:Num_DHSs] += 1
    list1 = [Num_samples, Num_DHSs, chosenthresh, cosdist_sample_thresh,  cosdist_DHS_thresh, density_in_recon, density_in_data, precision, recall]
    theheader =  ' '.join(str(e) for e in list1)
    namestr = ' '.join(names[Sample_appear_cut][Sample_closeness][0:Num_samples])
    theheader +=' '+namestr
    print(namestr)

doing cluster  0
cluster length  8502
name of samples ['31' '32' '33' ..., '142390' '142392' '142393']
(8502, 176)
 Num DHSs 35264
DHSs meeting threshold  [153690 153689 153688 ...,  55942 177333 131876]
close samples  [ 0.08435226]
 Num_samples 1
density in reconstruction,  density in data,  precision,  recall
0.7206215970961888 0.6240074863883848 0.7970643790335274 0.9204726198591229
DS26455A_h.iPSC_LN26496
doing cluster  1
cluster length  5553
name of samples ['2226' '2227' '2244' ..., '142728' '142825' '142826']
(5553, 176)
 Num DHSs 31503
DHSs meeting threshold  [44285 44286 44289 ..., 90960 90961 66598]
close samples  [ 0.00450358  0.01513161  0.0321376   0.02027089  0.65312444  0.00170648
  0.00677009  0.00181057  0.01660937  0.01040684  0.60252754  0.01998297
  0.04710005  0.08557821  0.62166314  0.31769092]
 Num_samples 13
density in reconstruction,  density in data,  precision,  recall
0.5095045893065129 0.42013825301131275 0.7239746575802015 0.877969115963339
DS17189_CD56+_L